In [ ]:
import pandas as pd

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Clustering

# ## Problem Statement

# Students have to evaluate a lot of factors before taking a decision to join a university for their higher education requirements.
# The objective of this project is to simplify the process of applying for appropriate universities which are of similar nature.

# `CRISP-ML(Q)` process model describes six phases:
#
# 1. Business and Data Understanding
# 2. Data Preparation
# 3. Model Building
# 4. Model Evaluation
# 5. Model Deployment
# 6. Monitoring and Maintenance

# **Objective(s):** Maximize the convenience of admission process
#
# **Constraints:** Minimize the brain drain
#

# **Success Criteria**
#
# - **Business Success Criteria**: Reduce the application process time from anywhere between 20% to 40%
#
# - **ML Success Criteria**: Achieve Silhoutte coefficient of atleast 0.6
#
# - **Economic Success Criteria**: US Higher education department will see an increase in revenues by atleast 30%
#
# **Proposed Plan:**
# Grouping the available universities will allow to understand the characteristics of each group.

# In[ ]:





# ## Data Collection

# Data:
#    The university details are obtained from the US Higher Education Body and is publicly available for students to access.
#
# Data Dictionary:
# - Dataset contains 25 university details
# - 7 features are recorded for each university
#
# Description:
# - Univ - University Name
# - State - Location (state) of the university
# - SAT - Average SAT score for eligibility
# - Top10 - % of students who ranked in top 10 in their previous academics
# - Accept - % of students admitted to the universities
# - SFRatio - Student to Faculty ratio
# - Expenses - Overall cost in USD
# - GradRate - % of students who graduate

# In[ ]:


get_ipython().system('pip install feature_engine')


# **Importing required packages**

# In[1]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from feature_engine.outliers import Winsorizer


# **Import the data**

# In[2]:


df = pd.read_excel(r'C:\Users\Bharani Kumar\Downloads\University_Clustering (3).xlsx')
df


# In[3]:


df.info()


# ## EXPLORATORY DATA ANALYSIS (EDA) / DESCRIPTIVE STATISTICS

# ***1st Moment Business Decision (Measures of Central Tendency)***
#
# 1) Mean
#
# 2) Median
#
# 3) Mode

# In[4]:


print(df.mean())
print('\n')
print(df.median())
print('\n')
print(df.mode())


# ***2nd Moment Business Decision (Measures of Dispersion)***
#
# 1) Variance
#
# 2) Standard deviation
#
# 3) Range (maximum - minimum)

# In[5]:


print(df.var())
print('\n')
print(df.std())


# ***3rd Business Moment Decision (Skewness)***
#
# Measure of asymmetry in the data distribution

# In[6]:


df.skew()


# ***4th Business Moment Decision (Kurtosis)***
#
# Measure of peakedness - represents the overall spread in the data

# In[7]:


df.kurt()


# ***Descriptive Statistics and Data Distribution Function***

# In[8]:


df.describe()


# In[ ]:





# ## Data Preprocessing and Cleaning

# **Typecasting** :
#
# As Python automatically interprets the data types, there may be a requirement for the data type to be converted. The process of converting one data type to another data type is called Typecasting.
#
# Example:
# 1) int to float
#
# 2) float to int

# In[9]:


df.dtypes


# The dataset contains UnivID which is being interpreted as Integer by Python. UnivID is an identity and a unique number given to each university. Hence it should be treated as a categorical data. We can convert the integer data to string type.

# In[10]:


# Convert 'int64' to 'str' (string) type.

df.UnivID = df.UnivID.astype('str')
df.dtypes


# **Cleaning Unwanted columns**
#
# UnivID is an identity to each university. Analytically it does not have any value (Nominal data). We can safely ignore the ID column by dropping the column.

# In[11]:


df.drop(['UnivID'], axis = 1, inplace = True)


# In[12]:


df.info()


# In[ ]:





# **Handling duplicates:**
#
# If the dataset has multiple entries of the same record then we can remove the duplicate entries. In our dataset we don't have duplicates. In case of duplicates we will use function drop_duplicates()

# In[13]:


duplicate = df.duplicated()  # Returns Boolean Series denoting duplicate rows.
print(duplicate)
sum(duplicate)


# In[14]:


# Removing Duplicates
df = df.drop_duplicates() # Returns DataFrame with duplicate rows removed.


# **Missing Value Analysis**
#
# ***IMPUTATION:***
#
# The process of dealing with missing values is called Imputation.
#
# Most popular substitution based Imputation techniques are:
#
# 1) Mean imputation for numeric data
#
# 2) Mode imputation for non-numeric data

# In[15]:


df.isnull().sum() # Check for missing values


# In[16]:


type(df["SAT"])


# In[17]:


type(df[["SAT"]])


# In[18]:


from sklearn.impute import SimpleImputer

# Mean Imputer
mean_imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
df["SAT"] = pd.DataFrame(mean_imputer.fit_transform(df[["SAT"]]))

df["SAT"].isna().sum()


# In[19]:


# Median Imputer
median_imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
df["SFRatio"] = pd.DataFrame(median_imputer.fit_transform(df[["SFRatio"]]))

df["SFRatio"].isna().sum()  # all records replaced by median


# In[20]:


# Random Imputer
from feature_engine.imputation import RandomSampleImputer

random_imputer = RandomSampleImputer(['GradRate'])
df["GradRate"] = pd.DataFrame(random_imputer.fit_transform(df[["GradRate"]]))

df["GradRate"].isna().sum()  # all records replaced by a random value


# In[21]:


df.isna().sum()


# ## Single code for Mean Imputation on the entire dataset.
# #### Alternative option

# In[22]:


df.fillna(df.mean(), inplace = True)


# **Outliers Analysis**:
#
# Extreme data values in a variable can be outliers. In case of outliers we can use one of the strategies of 3 R (Rectify, Retain, or Remove)

# **Dividing Categorical and Numerical variables from dataset**

# In[23]:


df_cate = df.iloc[:, :2]
df_cate.head()


# In[24]:


df_num = df.iloc[:, 2:]
df_num.head()


# **Box Plot**
#
# Visualize numeric data using boxplot for outliers

# In[25]:


# Multiple boxplots in a single visualization.
# Columns with larger scales affect other columns.
# Below code ensures each column gets its own y-axis.

# pandas plot() function with parameters kind = 'box' and subplots = True

df_num.plot(kind = 'box', subplots = True, sharey = False, figsize = (10, 6))

'''sharey True or 'all': x- or y-axis will be shared among all subplots.
False or 'none': each subplot x- or y-axis will be independent.'''


# increase spacing between subplots
plt.subplots_adjust(wspace = 0.75) # ws is the width of the padding between subplots, as a fraction of the average Axes width.
plt.show()


# Boxplots show outliers in: SAT, Top10, Accept, SFRatio.
#
# ***Outlier Analysis***:
# We shall use Winsorization Technique to treat outliers
#
# Winsorization function rounds off the exceptional data points based on capping method used in parameters/hyperparameters section.

# In[26]:


# D-Tale
########

# pip install dtale
import dtale

d = dtale.show(df)
d.open_browser()


# #### Install the required package if it is not already available
# ***Package to get winsorization function***
#
# !pip install feature_engine

# In[27]:


# Winsorization for "SAT, Top10, Accept, SFRatio" column
winsor = Winsorizer(capping_method = 'iqr', # choose IQR rule boundary
                          tail = 'both', # cap left, right or both tails
                          fold = 1.5,
                          variables = ['SAT'])

df_num['SAT'] = winsor.fit_transform(df_num[['SAT']]) # this is replacing the outliers to cap values


# In[28]:


# Winsorization for "Top10" column
winsor = Winsorizer(capping_method = 'iqr', # choose IQR rule boundary
                          tail = 'both', # cap left, right or both tails
                          fold = 1.5,
                          variables = ['Top10'])

df_num['Top10'] = winsor.fit_transform(df_num[['Top10']]) # this is replacing the outliers to cap values


# In[29]:


# Winsorization for "Accept" column
winsor = Winsorizer(capping_method = 'iqr', # choose IQR rule boundary
                          tail = 'both', # cap left, right or both tails
                          fold = 1.5,
                          variables = ['Accept'])

df_num['Accept'] = winsor.fit_transform(df_num[['Accept']]) # this is replacing the outliers to cap values


# In[30]:


# Winsorization for "SFRatio" column
winsor = Winsorizer(capping_method = 'iqr', # choose IQR rule boundaries
                          tail = 'both', # cap left, right or both tails
                          fold = 1.5,
                          variables = ['SFRatio'])

df_num['SFRatio'] = winsor.fit_transform(df_num[['SFRatio']]) # this is replacing the outliers to cap values


# #### Verify for outliers post the treatment

# In[31]:


df_num.plot(kind = 'box', subplots = True, sharey = False, figsize = (10, 6)) # sharey = True shares the Y axis across all plots


# increase spacing between subplots
plt.subplots_adjust(wspace = 0.75)
plt.show()


# **Zero and near zero variance**
#
#    We dont have any values with near zero (or) zero variance. We can proceed with further data preprocessing steps.

# In[32]:


df_num.var()


# **Discretization / Binning / Grouping:**
#
# The process of converting continuous data into discrete number of bins.
#
# The current business requirement does not justify the discretization/binning requirement for any column.

# ### Dummy Variable Creation

# In[33]:


df.dtypes  #Verify the data types of the fields for the entire data


# In[34]:


# Alternatively we can use:

df.info()


# In[35]:


df_cate.info()


# In[36]:


print(df_cate.State.unique())
print('\n')

print(df_cate.State.value_counts())


# - Univ name is an identity and does not require to be converted into numeric values.
#
# - STATE column is an important field in decision making hence it is to be converted into Numeric values.

# In[37]:


# Applying One-Hot Encoding with get_dummies

df_cate = pd.get_dummies(df_cate, columns = ['State'], drop_first = True)


# In[38]:


df_cate.info()


# In[39]:


# The dimension of the dataset after One-Hot Encoding

df_cate.shape


# In[40]:


df_cate.head()


# In[ ]:





# # **Graphical Representation**

# In[2]:


pip install PyQt5 pyqtwebengine --user # run this if you get any warning msg and if the plots are not visible


# In[85]:


get_ipython().run_line_magic('matplotlib', 'inline')


# In[86]:


df_num.columns


# In[52]:


# Histogram describes the spread of data

for i in df_num.columns:
  plt.hist(x = df_num[i])
  plt.title("Histogram for " + str(i))
  plt.show()


# In[53]:


# Normal Quantile-Quantile Plot - Helps in understanding the distribution of the data - Normal distribution or not
import scipy.stats as stats
import pylab

# Checking whether data is normally distributed
stats.probplot(df_num.SAT, dist = "norm", plot = pylab)


# In[87]:


# Checking whether data is normally distributed
stats.probplot(df_num.Top10, dist = "norm", plot = pylab)


# In[88]:


# Checking whether data is normally distributed
stats.probplot(df_num.Accept, dist = "norm", plot = pylab)


# In[89]:


# Checking whether data is normally distributed
stats.probplot(df_num.SFRatio, dist = "norm", plot = pylab)


# In[90]:


# Checking whether data is normally distributed
stats.probplot(df_num.Expenses, dist = "norm", plot = pylab)


# In[91]:


# Checking whether data is normally distributed
stats.probplot(df_num.GradRate, dist = "norm", plot = pylab)


# ### Concatenate the Entire Data

# In[92]:


df1 = pd.concat([df_cate, df_num], axis = 1)  # Preprocessed data for clustering
df1.head()


# In[55]:


df1.shape


# In[94]:


df1.info()


# ## Bivariate Analysis

# **Scatter Plot**

# In[95]:


plt.scatter(x = df1['SAT'], y = df1['Top10'])


# In[96]:


sns.pairplot(df_num)   # Use numerical variables for scatter plots


#
# **Heatmap**

# In[98]:


corrmatrix = df_num.corr(method = "pearson")

sns.heatmap(corrmatrix, xticklabels = corrmatrix.columns, yticklabels = corrmatrix.columns, cmap = "coolwarm")
plt.title('Heat Map of Correlation Matrix')
plt.tight_layout()
plt.show()


# ### 9) Scaling using Normalization:
#
# The scale/magnitude of the data will be converted to min = 0 and max = 1

# In[99]:


def norm_func(i):
  x = (i-i.min()) / (i.max()-i.min())
  return(x)


# In[100]:


df_norm = norm_func(df1.iloc[:, 1:])


# In[101]:


df_norm.describe()


# # CLUSTERING MODEL BUILDING

# ### Hierarchical Clustering - Agglomerative Clustering

# In[102]:


# Libraries for creating dendrogram
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt


# In[66]:


plt.figure(1, figsize = (16, 8))
tree_plot = dendrogram(linkage(df_norm, method  = "ward"))

plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Index')
plt.ylabel('Euclidean distances')
plt.show()


# In[104]:


# Applying AgglomerativeClustering choosing 3 as clusters from the above dendrogram
hc1 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'complete')

y_hc1 = hc1.fit_predict(df_norm)
y_hc1


# In[105]:


# Applying AgglomerativeClustering choosing 3 as clusters from the above dendrogram
hc2 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'average')

y_hc2 = hc2.fit_predict(df_norm)
y_hc2


# In[106]:


# Applying AgglomerativeClustering choosing 3 as clusters from the above dendrogram
hc3 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'single')

y_hc3 = hc3.fit_predict(df_norm)
y_hc3


# In[107]:


hc1.labels_   # Refering to the results obtained from linkage = 'complete' method


# In[108]:


cluster_labels = pd.Series(hc1.labels_)


# In[109]:


# Add Cluster labels to orignal Dataset

df['cluster'] = cluster_labels
df.head()


# # Cluster Evaluation

# In[110]:


df.head() # The final dataframe with cluster ID's


# In[74]:


df2 = df.sort_values(by = 'cluster', ascending = True)
df2


# In[75]:


# Aggregate by doing mean of each cluster
df2.iloc[:, 2:9].groupby(df2.cluster).mean()


# In[76]:


cluster0 = df2.loc[(df2.cluster == 0), :]
cluster0


# In[111]:


cluster1 = df2.loc[(df2.cluster == 1), :]
cluster1


# In[112]:


cluster2 = df2.loc[(df2.cluster == 2), :]
cluster2


# In[113]:


cluster0.to_csv('University_0.csv', encoding = 'utf-8')
import os
os.getcwd()


# In[114]:


cluster1.to_csv('University_1.csv', encoding = 'utf-8')
import os
os.getcwd()


# In[115]:


cluster2.to_csv('University_2.csv', encoding = 'utf-8')
import os
os.getcwd()


# ## Another way of evaluating clusters is using outliers

# In[82]:


for i in cluster0.columns[2:8]:
  sns.boxplot(x = cluster0[i])
  plt.show()


# In[83]:


for i in cluster1.columns[2:8]:
  sns.boxplot(x = cluster1[i])
  plt.show()


# In[84]:


for i in cluster2.columns[2:8]:
  sns.boxplot(x = cluster2[i])
  plt.show()


# In[ ]:






SyntaxError: ignored